In [1]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import os
import re
from IPython.display import clear_output

------
<a id="indice"></a>

# Índice

1. **[Coleta](#coleta)**
    * [Twitter Archive](#coleta:twitter-archive)
    * [Image Prediction](#coleta:image-prediction)
    * [Twitter API](#coleta:twitter-api)
    
    
2. **[Avaliação](#avaliacao)**
    * [Twitter Archive](#avaliacao:twitter-archive)
    * [Image Prediction](#avaliacao:image-prediction)
    * [Twitter API](#avaliacao:twitter-api)
    * [Notas da avaliação](#avaliacao:anotacoes)
    
    
3. **[Limpeza](#limpeza)**
    * [Definição](#limpeza:definicao)
    * [Twitter Archive](#limpeza:twitter-archive)
    * [Image Prediction](#limpeza:image-prediction)
    * [Twitter API](#limpeza:twitter-api)


4. **[Armazenamento](#armazenamento)**

------
<a id="coleta"></a>

# Coleta

<a id="coleta:twitter-archive"></a>

## Coleta: Twitter archive

In [2]:
df_twitter_arc = pd.read_csv('data/twitter-archive-enhanced.csv')
df_twitter_arc.sample(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
790,773922284943896577,NaN,NaN,2016-09-08 16:33:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Heinrich (pronounced ""Pat""). He's a Bo...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/773922284...,12,10,Heinrich,None,None,None,None
1086,738166403467907072,NaN,NaN,2016-06-02 00:32:39 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Axel. He's a professional leaf catcher...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/738166403...,12,10,Axel,None,None,None,None


<a id="coleta:image-prediction"></a>

## Coleta: Image prediction

In [3]:
df_prediction = None

r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

if r.status_code is 200:
    df_prediction = pd.read_csv(pd.compat.StringIO(r.text), sep='\t')    
else:    
    print('ERROR: Image prediction request returned {status_code} status code.'.format(status_code = r.status_code))

In [4]:
df_prediction.sample(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
123,668226093875376128,https://pbs.twimg.com/media/CUYEFlQXAAUkPGm.jpg,1,trombone,0.390339,False,cornet,0.314149,False,French_horn,0.255182,False
1003,708834316713893888,https://pbs.twimg.com/media/CdZI_bpWEAAm1fs.jpg,1,Eskimo_dog,0.283945,True,giant_panda,0.218252,False,malamute,0.180401,True


<a id="coleta:twitter-api"></a>

## Coleta: Twitter API

In [5]:
# Carregar configuração do Twitter App
with open('twitter_config.json', 'r', encoding='utf-8') as file:
    app_config = json.load(file)

In [6]:
# Atribuição da configurações a variaveis locais
api_key = app_config['api_key']
api_secret = app_config['api_secret']
access_token = app_config['access_token']
access_secret = app_config['access_secret']

In [7]:
# Autentificação ao Twitter App
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

Buscar dados do histórico de Tweets na API

In [8]:
json_path = 'data/tweet_json.txt'
error_log = 'data/tweet_error.log'

# Salvar Tweets localmente
if(not os.path.isfile(json_path)):
    print('Aguarde. Dado o volume de requisições, a coleta de tweets pode demorar alguns minutos.')
    tweet_count = 0
    for tweet_id in df_twitter_arc.tweet_id:
        print('{percent}%'.format(percent=int((++tweet_count/df_twitter_arc.tweet_id.size)*100)))
        try:
            status = api.get_status(tweet_id)
            with open(json_path, 'a', newline='\n') as file:
                file.write(f'{json.dumps(status._json)}\n')

        except Exception as err:
            with open(error_log, 'a', newline='\n') as log:
                log.write(f'{str(tweet_id)}: {str(err.args[0])}\n')
            print(f'{str(tweet_id)}: {str(err.args[0])}')
        
        clear_output(wait=True)
else:
    print('Dados já salvos em disco, não serão executadas novas requisições à API do Twitter.')
        

Dados já salvos em disco, não serão executadas novas requisições à API do Twitter.


In [9]:
# Criar uma lista de Dictionaries com os Tweets carregados
tweets = []

with open(json_path, 'r') as file:
    for line in file:
        try:
            tweet = json.loads(line)
            
            if(tweet.get('entities', False)):
                if(tweet['entities'].get('media', False) and tweet['entities']['media'][0].get('media_url', False)):
                    tweets.append({
                        'id': tweet['id'],
                        'created_at': tweet['created_at'],
                        'in_reply_to_status_id': tweet['in_reply_to_status_id'],
                        'in_reply_to_user_id': tweet['in_reply_to_user_id'],
                        'is_quote_status': tweet['is_quote_status'],
                        'retweet_count': tweet['retweet_count'],
                        'favorite_count': tweet['favorite_count'],
                        'media_url': tweet['entities']['media'][0]['media_url'],
                        'retweeted': tweet['retweeted'],
                        'favorited': tweet['favorited']
                    })
            
            
        except Exception as e:
            print(e) 
            
        

In [10]:
# Criar dataframe de Tweets consultados na API
columns = tweets[0].keys()
df_tweets_api = pd.DataFrame(tweets, columns = columns)

------
<a id="avaliacao"></a>

# Avaliação

<a id="avaliacao:twitter-archive"></a>

## Avaliação: Twitter data archive

In [11]:
 df_twitter_arc.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1163,723673163800948736,NaN,NaN,2016-04-23 00:41:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Ivar. She is a badass Viking warrior. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/723673163...,10,10,Ivar,None,None,None,None
1027,746056683365994496,NaN,NaN,2016-06-23 19:05:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Arlen and Thumpelina. They are best pa...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/746056683...,11,10,Arlen,None,None,None,None
1465,694352839993344000,NaN,NaN,2016-02-02 02:53:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Oliviér. He takes killer selfies. Has a d...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/694352839...,10,10,Oliviér,None,None,None,None
936,753375668877008896,NaN,NaN,2016-07-13 23:48:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Hank. He's mischievous af. Doesn't eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/753375668...,8,10,Hank,None,None,None,None
1553,688908934925697024,NaN,NaN,2016-01-18 02:21:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Clarence. He does parkour. 8/10 very tale...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/688908934...,8,10,Clarence,None,None,None,None


In [12]:
df_twitter_arc.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [13]:
df_twitter_arc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

<a id="twitter-arc:info"></a>
* Dados incompletos, contém apenas 2356 registros dos 5000 anunciados
* `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id` e `retweeted_status_user_id` em `float`
* `timestamp` e `retweeted_status_timestamp` em `string`

In [14]:
# Verificar se há IDs duplicados
df_twitter_arc[df_twitter_arc.tweet_id.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


<a id="twitter-arc:source"></a>

In [15]:
# Visualizar exemplos de dados da coluna `source`
df_twitter_arc.loc[:, 'source'].sample(10)

2105    <a href="http://twitter.com/download/iphone" r...
1285    <a href="http://twitter.com/download/iphone" r...
1283    <a href="http://twitter.com/download/iphone" r...
269     <a href="http://twitter.com/download/iphone" r...
1516    <a href="http://twitter.com/download/iphone" r...
558     <a href="http://twitter.com/download/iphone" r...
493     <a href="http://twitter.com/download/iphone" r...
1389    <a href="http://twitter.com/download/iphone" r...
2091    <a href="http://twitter.com/download/iphone" r...
1858    <a href="http://twitter.com/download/iphone" r...
Name: source, dtype: object

Variável `source` é uma URL para a Aplicação utilizada no tweet, sem valor para a unidade de observação

<a id="twitter-arc:name"></a>

In [16]:
# Visualizar exemplos da coluna `name`
df_twitter_arc.name.sample(10)

1228         None
1617      Gerbald
2304            a
676          None
1218       Curtis
390          None
1126        Ollie
45          Bella
799     Roosevelt
294       Waffles
Name: name, dtype: object

In [17]:
df_twitter_arc[df_twitter_arc.name == 'None'].name.count()

745

* Nomes (`name`) preenchidos com valor literal igual a `'None'`, supostamente um valor não preenchido, vazio

<a id="twitter-arc:category"></id>

In [18]:
# Visualizar exemplo de dados das colunas 'doggo' a 'puppo'
df_twitter_arc.loc[:,'doggo':].sample(10)

,doggo,floofer,pupper,puppo
1643,None,None,None,None
643,None,None,None,puppo
1468,None,None,None,None
823,None,None,None,None
1799,None,None,None,None
1947,None,None,None,None
2262,None,None,None,None
155,None,None,None,None
1019,None,None,None,None
2103,None,None,None,None


* Valores não preenchidos como 'None' literal

Uma variável em várias colunas

In [19]:
# Verificar padrão no preenchimento das classificações
df_twitter_arc.loc[:,'doggo':].nunique()

doggo      2
floofer    2
pupper     2
puppo      2
dtype: int64

In [20]:
df_twitter_arc.loc[:, 'doggo':'puppo'].isna().sum()

doggo      0
floofer    0
pupper     0
puppo      0
dtype: int64

<a id="twitter-arc:expanded_urls"></a>

Exemplos da coluna `expanded_urls`

In [21]:
df_twitter_arc.loc[:, 'expanded_urls'].sample(20)

276     https://www.gofundme.com/3hgsuu0,https://twitt...
466     https://twitter.com/dog_rates/status/817171292...
1361    https://twitter.com/dog_rates/status/703079050...
2259    https://twitter.com/dogratingrating/status/667...
1312    https://twitter.com/dog_rates/status/707014260...
503     https://twitter.com/dog_rates/status/813066809...
2019    https://twitter.com/dog_rates/status/672125275...
1836    https://twitter.com/dog_rates/status/676098748...
156     https://twitter.com/dog_rates/status/861383897...
402     https://twitter.com/dog_rates/status/824325613...
2213    https://twitter.com/dog_rates/status/668567822...
925     https://twitter.com/dog_rates/status/755110668...
2335    https://twitter.com/dog_rates/status/666287406...
2252    https://twitter.com/dog_rates/status/667801013...
2080    https://twitter.com/dog_rates/status/670823764...
809     https://twitter.com/dog_rates/status/771500966...
1219    https://twitter.com/dog_rates/status/714631576...
2110    https:

In [22]:
df_twitter_arc.loc[:, 'expanded_urls'].isna().value_counts()

False    2297
True       59
Name: expanded_urls, dtype: int64

* Registros de `expanded_urls` com URLs para os twittes originais. Não apresenta valor a unidade em observacão

------
<a id="avaliacao:image-prediction"></a>

## Avaliação: Image prediction

In [23]:
df_prediction.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


<a id="image-prediction:info"></info>

In [24]:
df_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [25]:
df_prediction.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1810,832769181346996225,https://pbs.twimg.com/media/C46UmzSVMAAqBug.jpg,1,jersey,0.895698,False,sweatshirt,0.089085,False,poncho,0.002975,False
420,674042553264685056,https://pbs.twimg.com/media/CVquIDRW4AEJrPk.jpg,1,toy_poodle,0.927975,True,miniature_poodle,0.068946,True,standard_poodle,0.001316,True
567,678341075375947776,https://pbs.twimg.com/media/CWnznDTU4AAa-6P.jpg,1,golden_retriever,0.853284,True,cocker_spaniel,0.026230,True,Labrador_retriever,0.024123,True
360,672640509974827008,https://pbs.twimg.com/media/CVWy9v-VAAALSoE.jpg,1,Chesapeake_Bay_retriever,0.420155,True,Cardigan,0.266030,True,Labrador_retriever,0.042514,True
1125,727314416056803329,https://pbs.twimg.com/media/Chfwmd9U4AQTf1b.jpg,2,toy_poodle,0.827469,True,miniature_poodle,0.160760,True,Tibetan_terrier,0.001731,True
1202,741303864243200000,https://pbs.twimg.com/media/Ckmj7mNWYAA4NzZ.jpg,1,Chihuahua,0.768156,True,pug,0.014902,True,Pekinese,0.012816,True
1160,734776360183431168,https://pbs.twimg.com/media/CjJzMlBUoAADMLx.jpg,1,Siberian_husky,0.304902,True,Eskimo_dog,0.155147,True,malamute,0.050942,True
280,671115716440031232,https://pbs.twimg.com/media/CVBILUgVAAA1ZUr.jpg,1,malinois,0.406341,True,kelpie,0.143366,True,dingo,0.129802,False
741,687317306314240000,https://pbs.twimg.com/media/CYnXcLEUkAAIQOM.jpg,1,Shih-Tzu,0.747208,True,Maltese_dog,0.091025,True,Lhasa,0.035788,True
961,705898680587526145,https://pbs.twimg.com/media/CcvbGj5W8AARjB6.jpg,1,collie,0.808276,True,Border_collie,0.059437,True,groenendael,0.026720,True


<a id="image-prediction:p-values"></a>

Valores não padronizados para as variáveis `p1`, `p2` e `p3`

In [26]:
# Visualização de exemplos de valores nas colunas `p1`, `p2` e `p3`
df_prediction.loc[:, ['p1', 'p2', 'p3']].sample(5)

,p1,p2,p3
1446,golden_retriever,Labrador_retriever,doormat
1328,seat_belt,golden_retriever,Tibetan_mastiff
1703,Tibetan_mastiff,Tibetan_terrier,otterhound
1615,Chihuahua,toy_terrier,kelpie
121,Labrador_retriever,golden_retriever,Chesapeake_Bay_retriever


<a id="image-prediction:duplicated-urls"></a>

Verificar a duplicidade de Tweet ID (`tweet_id`) e Imagens (`jpg_url`)

In [27]:
# Verificar a duplicidade de IDs
df_prediction.tweet_id.nunique()

2075

Verificar a duplicidade na coluna `jpg_url`

In [28]:
df_prediction.jpg_url.duplicated().value_counts()

False    2009
True       66
Name: jpg_url, dtype: int64

Há 66 imagens duplicadas

In [29]:
# Validar indicadores de confiabiliade da predição
df_prediction.loc[:, ['p1_conf', 'p2_conf', 'p3_conf']].max()

p1_conf    1.000000
p2_conf    0.488014
p3_conf    0.273419
dtype: float64

------
<a id="avaliacao:twitter-api"></a>

## Avaliação: Twitter API Requests

In [30]:
df_tweets_api.describe()

,id,in_reply_to_status_id,in_reply_to_user_id,retweet_count,favorite_count
count,1.820000e+03,2.200000e+01,2.200000e+01,1820.000000,1820.000000
mean,7.237270e+17,6.992047e+17,4.196984e+09,2512.144505,6820.872527
std,5.777841e+16,4.409222e+16,0.000000e+00,4891.076169,11900.984651
min,6.660209e+17,6.671522e+17,4.196984e+09,11.000000,0.000000
25%,6.747671e+17,6.747625e+17,4.196984e+09,536.000000,1363.500000
50%,7.008223e+17,6.799651e+17,4.196984e+09,1118.000000,3153.000000
75%,7.617466e+17,7.032024e+17,4.196984e+09,2526.250000,7463.250000
max,8.924206e+17,8.558181e+17,4.196984e+09,82905.000000,163034.000000


<a id="tweets-api:info"></a>

In [31]:
df_tweets_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1820 entries, 0 to 1819
Data columns (total 10 columns):
id                       1820 non-null int64
created_at               1820 non-null object
in_reply_to_status_id    22 non-null float64
in_reply_to_user_id      22 non-null float64
is_quote_status          1820 non-null bool
retweet_count            1820 non-null int64
favorite_count           1820 non-null int64
media_url                1820 non-null object
retweeted                1820 non-null bool
favorited                1820 non-null bool
dtypes: bool(3), float64(2), int64(3), object(2)
memory usage: 104.9+ KB


Os nomes das colunas `id` e `media_url` estão fora do padrão dos demais dataframes, respectivamente `tweet_id` e `jpg_url` 

In [32]:
df_tweets_api.sample(5)

,id,created_at,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,retweet_count,favorite_count,media_url,retweeted,favorited
1102,685667379192414208,Sat Jan 09 03:40:16 +0000 2016,NaN,NaN,False,603,2441,http://pbs.twimg.com/media/CYP62A6WkAAOnL4.jpg,False,False
897,702321140488925184,Wed Feb 24 02:36:23 +0000 2016,NaN,NaN,False,1087,3410,http://pbs.twimg.com/media/Cb8lWajW0AEDOsY.jpg,False,False
517,753294487569522689,Wed Jul 13 18:26:16 +0000 2016,NaN,NaN,False,1116,3551,http://pbs.twimg.com/media/CnQ9Vq1WEAEYP01.jpg,False,False
1173,681679526984871937,Tue Dec 29 03:33:58 +0000 2015,NaN,NaN,False,476,1838,http://pbs.twimg.com/media/CXXP5O4WEAA4dgS.jpg,False,False
1311,676098748976615425,Sun Dec 13 17:57:57 +0000 2015,NaN,NaN,False,1499,3078,http://pbs.twimg.com/media/CWH8L72UkAAvjql.jpg,False,False


* `created_at` como `string`
* `in_reply_to_status_id` e `in_reply_to_userid` como `Float`

<a id="tweets-api:duplicated"></a>
Verificar a duplicidade de valores

In [33]:
# Verificar se há IDs duplicados
df_tweets_api.id.duplicated().value_counts()

False    1820
Name: id, dtype: int64

In [34]:
# Verificar se há imagens duplicadas
df_tweets_api.media_url.duplicated().value_counts()

False    1759
True       61
Name: media_url, dtype: int64

Há tweets referenciando a mesma imagem

------

<a id="avaliacao:anotacoes"></a>

## Notas da avaliação
### Qualidade

#### `df_twitter_arc` - Twitter archive
1. [Base incompleta, contém apenas **2356** registros dos **5000** anunciados](#twitter-arc:info)
* [Coluna `source` sem valor à unidade de obsevação](#twitter-arc:source)
* [Nomes (`name`) preenchidos com valor literal igual a `'None'`, vazio](#twitter-arc:name)
* [Coluna `expanded_urls` sem valor à unidade de observação](#twitter-arc:expanded_urls)
* [Categorias \[`doggo`, `floofer`, `pupper`, `puppo`\] com literal `'None'`](#twitter-arc:category)


#### `df_prediction` - Image prediction
1. [Previsões \[`p1`, `p2`, `p3`\] com nomes não padronizados](#image-prediction:p-values)
* [Duplicidade na coluna `jgp_url`](#image-prediction:duplicated-urls)


#### `df_tweets_api` - Twitter API
* [Diferentes Tweets referenciando a mesma imagem, duplicidade](#tweets-api:duplicated)


### Organização

#### `df_twitter_arc` - Twitter archive
1. [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#twitter-arc:info)
* [As colunas `timestamp` e `retweet_status_timestamp` estão em `string`](#twitter-arc:info)
* [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#twitter-arc:category)


#### `df_prediction` - Image prediction
1. [Colunas como variáveis na identificação das predições](#image-prediction:info)


#### `df_tweets_api` - Twitter API
1. [Os nomes das colunas `id` e `media_url` estão fora do padrão dos demais dataframes, respectivamente `tweet_id` e `jpg_url` ](#tweets-api:info)
* [Data de criação (`created_at`) como `string`](#tweets-api:info)
* [Valor do id na coluna `in_reply_to_status_id` como `float`](#tweets-api:info)
* [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#tweets-api:info)

------

<a id="limpeza"></a>

# Limpeza

<a id="limpeza:definicao"></a>

## Limpeza: Definição 

#### `df_twitter_arc` - Twitter archive
1. [Base incompleta, de **5000**, apenas **2356** estão disponíveis](#limpeza:twitter-arc:completeness)
    * OK - Nada a ser feito
* [Coluna `source` sem valor à unidade de observação](#limpeza:twitter-arc:source)
    * O conteúdo apresenta um link para download do App Twitter, portanto não agrega valor à unidade de observação, a coluna pode ser removida
* [Nomes (`name`) não preenchidos com valor literal igual a `'None'`](#limpeza:twitter-arc:name)
    * Alterar valores como 'None' para `np.nan`
* [Coluna `expanded_urls` sem valor à unidade de observação](#limpeza:twitter-arc:expanded_urls)(#twitter-arc:expanded_urls)
    * Não apresentam valor à unidade de observação, são URLs para os tweets em questão. Remover a coluna `expanded_urls`.
* [Categorias \[doggo, floofer, pupper, puppo\] com literal `'None'`](#limpeza:twitter-arc:category)
    * Aplicar valor `np.nan` para nomes iguais a 'None'   
* [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#limpeza:twitter-arc:info)
    * A coluna, `retweeted_status_id` ou `retweeted_status_user_id`, podem ser convertidas para `Boolean` como auxílio a identificação de tweets originais
    * Após a criação de uma coluna indicando se o tweet é um retweet, as colunas de retweet, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, podem ser descartadas
    * As colunas `in_reply_to_status_id` e `in_reply_to_user_id` serão retratadas no dataframe `df_tweets_api`
* [As colunas `timestamp` e `retweet_status_timestamp` estão em `string` e deveriam ser do tipo `datetime`](#limpeza:twitter-arc:datetime)
    * Converter valores para `datetime`
* [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#limpeza:twitter-arc:category-merge)
    * Reduzir a variável para uma coluna, `category`


#### `df_prediction` - Image prediction
1. [Previsões \[`p1`, `p2`, `p3`\] com nomes não padronizados](#limpeza:image-prediction:names)
    * Converter os valores para 'lower case' e substituir '_' por espaços
* [Previsões repetidas sobre a mesma imagem](#limpeza:image-prediction:duplicated-urls)
    * Descartar predições sobre imagens repetidas
* [Colunas como variáveis na identificação das predições](#limpeza:image-prediction:info)
    * Transpor as variáveis nas colunas para linhas


#### `df_tweets_api` - Twitter API
1. [Os nomes das colunas `id` e `media_url` estão fora do padrão dos demais dataframes, respectivamente `tweet_id` e `jpg_url` ](#limpeza:tweets-api:columns)
* [Diferentes Tweets referenciando a mesma imagem, duplicidade](#limpeza:tweets-api:duplicated)
* [Data de criação (`created_at`) como `string`](#limpeza:tweets-api:datetime)
    * Converter datas em string para o formato `datetime`
* [Valor do id na coluna `in_reply_to_status_id` e `in_reply_to_user_id` como `float`](#limpeza:tweets-api:info)
    * Converter para Inteiro as colunas `in_reply_to_status_id` e `in_reply_to_user_id` e atribuir np.nan as que estiverem sem valor
* [A coluna `media_url` está com o nome `jpg_url`nos demais dataframes](#limpeza:tweets-api:columns)

<a id="limpeza:twitter-archive"></a>

## Limpeza: Twitter archive

In [35]:
# Criar cópia do dataframe para a limpeza
df_arch_clean = df_twitter_arc.copy()

<a id="limpeza:twitter-arc:completeness"></a>

### 1. [Base incompleta, de 5000, apenas 2356 estão disponíveis](#limpeza:twitter-arc:completeness)
Nada a ser feito

In [36]:
df_arch_clean.shape

(2356, 17)

<a id="limpeza:twitter-arc:source"></a>

### 2. [Coluna `source` sem valor à unidade de observação](#limpeza:twitter-arc:source)
* O conteúdo apresenta um link para download do App Twitter, portanto não agrega valor à unidade de observação, a coluna pode ser removida

In [37]:
# Remoção da coluna `source`
df_arch_clean.drop('source', axis='columns', inplace=True)

#### Teste: Remoção coluna `source`

In [38]:
# Conferir que a coluna `source` foi removida
df_arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

<a id="limpeza:twitter-arc:name"></a>

### 3. [Nomes (`name`) não preenchidos com valor literal igual a `'None'`](#limpeza:twitter-arc:name)
    * Alterar valores como 'None' para `np.nan`

In [39]:
# Contagem de nomes iguais a 'None'
df_arch_clean[df_arch_clean.name.str.lower() == 'none'].name.count()

745

In [40]:
# Função para substituição de valores literais 'None' por np.nan
def set_nan_at_none(value):
    if str(value).lower() == 'none':
        return np.nan
    else:
        return value

In [41]:
# Aplicar `np.nan` aos nomes iguais a 'None'
df_arch_clean.name = df_arch_clean.name.apply(set_nan_at_none)

#### Teste: Aplicação de `np.nan` aos nomes como 'None'

In [42]:
df_arch_clean.name.isnull().sum()

745

In [43]:
df_arch_clean[df_arch_clean.name.str.lower() == 'none'].name.count()

0

In [44]:
df_arch_clean.name.sample(10)

1728      Rocco
2325     Walter
475        Jack
186         NaN
1305    Clarkus
1940        NaN
2089        NaN
1222     Travis
2346        the
871      Sephie
Name: name, dtype: object

<a id="limpeza:twitter-arc:expanded_urls"></a>

### 4. [Coluna `expanded_urls` sem valor à unidade de observação](#limpeza:twitter-arc:expanded_urls)
* Não apresentam valor à unidade de observação, são URLs para os tweets em questão. Remover a coluna `expanded_urls`

In [45]:
df_arch_clean.drop('expanded_urls', axis='columns', inplace=True)

#### Teste: Remoção da coluna `expanded_urls`

In [46]:
df_arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'rating_numerator', 'rating_denominator',
       'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

<a id="limpeza:twitter-arc:category"></a>

### 5. [Categorias \[`doggo`, `floofer`, `pupper`, `puppo`\] com literal `'None'`](#limpeza:twitter-arc:category)
* Aplicar valor `np.nan` para nomes iguais a 'None'   

In [47]:
# Trocar valores 'None' para np.nan nas colunas `doggo`, `floofer`, `pupper`, `puppo`
df_arch_clean.doggo = df_arch_clean.doggo.apply(set_nan_at_none)
df_arch_clean.doggo.isna().value_counts()

True     2259
False      97
Name: doggo, dtype: int64

In [48]:
df_arch_clean.floofer = df_arch_clean.floofer.apply(set_nan_at_none)
df_arch_clean.floofer.isna().value_counts()

True     2346
False      10
Name: floofer, dtype: int64

In [49]:
df_arch_clean.pupper = df_arch_clean.pupper.apply(set_nan_at_none)
df_arch_clean.pupper.isna().value_counts()

True     2099
False     257
Name: pupper, dtype: int64

In [50]:
df_arch_clean.puppo = df_arch_clean.puppo.apply(set_nan_at_none)
df_arch_clean.puppo.isna().value_counts()

True     2326
False      30
Name: puppo, dtype: int64

In [51]:
# Soma de valores preenchidos nas classificações
sum_categories = 0
sum_categories += df_arch_clean.doggo.notna().sum()
sum_categories += df_arch_clean.floofer.notna().sum()
sum_categories += df_arch_clean.pupper.notna().sum()
sum_categories += df_arch_clean.puppo.notna().sum()

sum_categories

394

Apenas 394 dos 2346 registros foram classificados

<a id="limpeza:twitter-arc:info"></a>

### 6. [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#limpeza:twitter-arc:info)
* Linhas com algum valor para as colunas `retweeted_status_id` ou `retweeted_status_user_id` podem ser descartadas por indicarem retweet, e o objetivo é analisar somente tweets originais
* Após a remoção retweets, as colunas `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, podem ser descartadas
* As colunas `in_reply_to_status_id` e `in_reply_to_user_id` serão retratadas no dataframe `df_tweets_api`

#### Identificação dos tweets originais
df_arch_clean.loc[:,'retweeted_status_id'].isna().value_counts()

In [52]:
df_arch_clean.retweeted_status_id.size

2356

In [53]:
df_arch_clean.loc[:,'retweeted_status_id'].isna().value_counts()

True     2175
False     181
Name: retweeted_status_id, dtype: int64

Dos 2356 registros, 181 são retweets

In [54]:
# Remoção dos retweets
df_arch_clean.drop(df_arch_clean[df_arch_clean.retweeted_status_id.notna()].index, axis=0, inplace=True)

#### Teste: Remoção de Retweets

In [55]:
# Consultar a existência de colunas com retweet_status_preenchido
df_arch_clean.loc[:,'retweeted_status_id'].isna().value_counts()

True    2175
Name: retweeted_status_id, dtype: int64

Remoção das colunas referentes a retweets

In [56]:
df_arch_clean.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis='columns', inplace=True)

#### Teste: Verificação da remoção das colunas referentes a retweets

In [57]:
df_arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'rating_numerator', 'rating_denominator', 'name', 'doggo',
       'floofer', 'pupper', 'puppo'],
      dtype='object')

<a id="limpeza:twitter-arc:datetime"></a>

### 7. [As colunas `timestamp` e `retweet_status_timestamp` estão em `string` e deveriam ser do tipo `datetime`](#limpeza:twitter-arc:datetime)
* Converter valores para `datetime`

Haja vista que a coluna `retweet_status_timestamp` foi removida no passo anterior, não será necessária a sua conversão

In [58]:
df_arch_clean.loc[:, ['timestamp']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 1 columns):
timestamp    2175 non-null object
dtypes: object(1)
memory usage: 34.0+ KB


In [59]:
df_arch_clean.timestamp = pd.to_datetime(df_arch_clean.timestamp)

#### Teste: Veriificação da conversão de `string` para `datetime`

In [60]:
df_arch_clean.loc[:, ['timestamp']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 1 columns):
timestamp    2175 non-null datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1)
memory usage: 34.0 KB


<a id="limpeza:twitter-arc:category-merge"></a>

### 8. [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#limpeza:twitter-arc:category-merge)
* Reduzir a variável para uma coluna, `category`

In [61]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null datetime64[ns, UTC]
text                     2175 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     1495 non-null object
doggo                    87 non-null object
floofer                  10 non-null object
pupper                   234 non-null object
puppo                    25 non-null object
dtypes: datetime64[ns, UTC](1), float64(2), int64(3), object(6)
memory usage: 220.9+ KB


In [62]:
df_arch_clean.loc[:, 'doggo':].notna().sum()

doggo       87
floofer     10
pupper     234
puppo       25
dtype: int64

In [63]:
df_arch_clean.loc[:, 'doggo':].notna().sum().sum()

356

De **2175** registros, apenas **356** estão classificados

Criar uma lista das categorias para mesclar as colunas, considerando a coluna a esquerda como prioritaria sobre valores nulos a direita.

In [64]:
category = df_arch_clean.doggo
category = category.combine_first(df_arch_clean.floofer)
category = category.combine_first(df_arch_clean.pupper)
category = category.combine_first(df_arch_clean.puppo)

df_arch_clean['category'] = category

In [65]:
df_arch_clean.category.notna().sum()

344

Após a unificação das colunas de classificação, pode-se ver que resultou em apenas **344** registros, diferente dos **356** iniciais. O que indica que haviam registros com mais de uma classificação.

In [66]:
df_arch_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis='columns', inplace=True)

#### Teste: Verificar a criação da coluna `category` a remoção das colunas `doggo`, `floofer`, `pupper` e `puppo`. 

In [67]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 9 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null datetime64[ns, UTC]
text                     2175 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     1495 non-null object
category                 344 non-null object
dtypes: datetime64[ns, UTC](1), float64(2), int64(3), object(3)
memory usage: 169.9+ KB


------
<a id="limpeza:image-prediction"></a>

## Limpeza: Image prediction 

In [68]:
# Cópia do dataframe para limpeza
df_predict_clean = df_prediction.copy()

<a id="limpeza:image-prediction:names"></a>

### 1. [Previsões \[`p1`, `p2`, `p3`\] com nomes não padronizados](#limpeza:image-prediction:names)
* Converter os valores para 'lower case' e substituir '_' por espaços

In [69]:
clean_prediction_name = lambda value: str(value).lower().replace('_', ' ')

In [70]:
df_predict_clean.loc[:, ['p1', 'p2', 'p3']].sample(5)

,p1,p2,p3
547,beagle,boxer,French_bulldog
796,golden_retriever,Labrador_retriever,Saluki
1946,home_theater,studio_couch,barber_chair
1628,Tibetan_mastiff,German_shepherd,Leonberg
1231,Pekinese,Shih-Tzu,cocker_spaniel


Aplicar a padronização dos nomes em cada uma das colunas de predição

In [71]:
df_predict_clean.p1 = df_predict_clean.p1.apply(clean_prediction_name)

In [72]:
df_predict_clean.p2 = df_predict_clean.p2.apply(clean_prediction_name)

In [73]:
df_predict_clean.p3 = df_predict_clean.p3.apply(clean_prediction_name)

#### Teste: Verificar padronização dos nomes das predições

In [74]:
df_predict_clean.loc[:, ['p1', 'p2', 'p3']].sample(5)

,p1,p2,p3
769,scotch terrier,affenpinscher,cairn
1241,labrador retriever,golden retriever,doormat
683,hog,mexican hairless,wild boar
881,brittany spaniel,welsh springer spaniel,beagle
1519,german shepherd,malinois,kelpie


<a id="limpeza:image-prediction:duplicated-urls"></a>

### 2. [Predições repetidas sobre a mesma imagem](#limpeza:image-prediction:duplicated-urls)
* Descartar predições sobre imagens repetidas

In [75]:
df_predict_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [76]:
df_predict_clean.loc[:, ['jpg_url', 'p1', 'p2', 'p3']].duplicated().value_counts()

False    2009
True       66
dtype: int64

Dos **2075** registros, **66** são sobre a mesma imagem. Entretanto, seguindo o objetivo de trabalhar com apenas Tweets originais, além de descartar duplicidade é interessante também descartar as refêrencias a Retweets. Para isso, será feita uma consulta a seleção de Tweets originais, que foi efetuada no passo [6 da limpeza do Twitter Archive](#limpeza:twitter-arc:info)

In [77]:
df_predict_clean = df_predict_clean.loc[df_predict_clean.tweet_id.isin(df_arch_clean.tweet_id)]

#### Teste: Seleção de Tweets originais

In [78]:
df_predict_clean.tweet_id.isin(df_arch_clean.tweet_id).value_counts()

True    1994
Name: tweet_id, dtype: int64

Apenas dos **2075**, **1994** são de Tweets originais

------

Dos Tweets originai ainda é necessário verificar a existência de duplicidade de predições.

In [79]:
df_predict_clean.loc[:, ['jpg_url']].duplicated().value_counts()

False    1994
dtype: int64

Nenhuma duplicidade foi encontrada após a remoção das referências de Retweets

In [80]:
df_predict_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    1994 non-null int64
jpg_url     1994 non-null object
img_num     1994 non-null int64
p1          1994 non-null object
p1_conf     1994 non-null float64
p1_dog      1994 non-null bool
p2          1994 non-null object
p2_conf     1994 non-null float64
p2_dog      1994 non-null bool
p3          1994 non-null object
p3_conf     1994 non-null float64
p3_dog      1994 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 161.6+ KB


<a id="limpeza:image-prediction:info"></a>

### 3. [Colunas como variáveis na identificação das predições](#limpeza:image-prediction:info)
* 3.1. Transpor as variáveis nas colunas para linhas:
    * 3.1.1. [`[p1, p2, p3]`, para `prediction_try`](#limpeza:image-prediction:try)
    * 3.1.2. [`[p1_conf, p2_conf, p3_conf]`, para `confidence`](#limpeza:image-prediction:conf)
    * 3.1.3. [`[p1_dog, p2_dog, p3_dog]`, para `is_a_dog`](#limpeza:image-prediction:dog)
* 3.2. [Unir os dataframes de prediction](#limpeza:image-prediction:merge)

In [81]:
df_predict_clean.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1060,715009755312439296,https://pbs.twimg.com/media/Cew5kyOWsAA8Y_o.jpg,1,dingo,0.310903,False,chihuahua,0.142288,True,cardigan,0.103945,True
1065,715680795826982913,https://pbs.twimg.com/media/Ce6b4MPWwAA22Xm.jpg,1,golden retriever,0.990715,True,labrador retriever,0.002228,True,chow,0.001197,True
124,668237644992782336,https://pbs.twimg.com/media/CUYOl0kW4AAVe_p.jpg,1,chow,0.809320,True,minivan,0.071311,False,pekinese,0.037870,True
523,676588346097852417,https://pbs.twimg.com/media/CWO5gmCUYAAX4WA.jpg,1,boston bull,0.976577,True,french bulldog,0.014324,True,chihuahua,0.002302,True
2052,887517139158093824,https://pbs.twimg.com/ext_tw_video_thumb/88751...,1,limousine,0.130432,False,tow truck,0.029175,False,shopping cart,0.026321,False


<a id="limpeza:image-prediction:try"></a>

#### 3.1.1. [`[p1, p2, p3]`, para `prediction_try`](#limpeza:image-prediction:try)

In [82]:
# Organizando as variáveis de tentativa de predição e seu palpite
df_predict_try = pd.melt(df_predict_clean,
        id_vars=['tweet_id', 'jpg_url'],
        value_vars=['p1', 'p2', 'p3'],
        var_name=['prediction_try'],
        value_name='prediction'
       )

In [83]:
df_predict_try.sample(5)

,tweet_id,jpg_url,prediction_try,prediction
5397,772152991789019136,https://pbs.twimg.com/media/Crc9DEoWEAE7RLH.jpg,p3,vizsla
2824,693647888581312512,https://pbs.twimg.com/media/CaBVE80WAAA8sGk.jpg,p2,doormat
741,687476254459715584,https://pbs.twimg.com/media/CYpoAZTWEAA6vDs.jpg,p1,wood rabbit
4609,680583894916304897,https://pbs.twimg.com/media/CXHrcFYWcAEE5_L.jpg,p3,hippopotamus
2176,669354382627049472,https://pbs.twimg.com/media/CUoGQjdXAAAkaz2.jpg,p2,french bulldog


Conversão dos valores da tentativa para número inteiro

In [84]:
# Metodo para retirar o índice da tentativa dos nomes das colunas
get_prediction_index = lambda value: int(re.sub(r'[^\d]', '', str(value)))

In [85]:
df_predict_try.prediction_try = df_predict_try.prediction_try.apply(get_prediction_index)

In [86]:
df_predict_try.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5982 entries, 0 to 5981
Data columns (total 4 columns):
tweet_id          5982 non-null int64
jpg_url           5982 non-null object
prediction_try    5982 non-null int64
prediction        5982 non-null object
dtypes: int64(2), object(2)
memory usage: 187.0+ KB


In [87]:
df_predict_try.sample(5)

,tweet_id,jpg_url,prediction_try,prediction
298,671488513339211776,https://pbs.twimg.com/media/CVGbPgrWIAAQ1fB.jpg,1,hermit crab
3719,829501995190984704,https://pbs.twimg.com/media/C4L7p19W8AA3Fs_.jpg,2,pekinese
4582,679527802031484928,https://pbs.twimg.com/media/CW4q7jDWkAA2y8g.jpg,3,cuirass
5835,856526610513747968,https://pbs.twimg.com/media/C-L-aIYXgAIR0jY.jpg,3,standard poodle
987,707969809498152960,https://pbs.twimg.com/media/CdM2xRpXEAUsR4k.jpg,1,toy poodle


<a id="limpeza:image-prediction:conf"></a>

#### 3.1.2. [`[p1_conf, p2_conf, p3_conf]`, para `confidence`](#limpeza:image-prediction:conf)

In [88]:
# Movendo o nível de confiança das predições para uma coluna apenas
df_predict_conf = pd.melt(df_predict_clean,
                         id_vars=['tweet_id', 'jpg_url'],
                         value_vars=['p1_conf', 'p2_conf', 'p3_conf'],
                         var_name=['prediction_try'],
                         value_name='confidence'
                        )

In [89]:
df_predict_conf.sample(5)

,tweet_id,jpg_url,prediction_try,confidence
2087,667549055577362432,https://pbs.twimg.com/media/CUOcVCwWsAERUKY.jpg,p2_conf,0.007737
2990,708469915515297792,https://pbs.twimg.com/media/CdT9n7mW0AQcpZU.jpg,p2_conf,0.127717
1751,834086379323871233,https://pbs.twimg.com/media/C5NFIsjWQAEI93t.jpg,p1_conf,0.736759
2504,676215927814406144,https://pbs.twimg.com/media/CWJmzNsWUAE706Z.jpg,p2_conf,0.000158
663,682750546109968385,https://pbs.twimg.com/media/CXmd_bsWkAEEXck.jpg,p1_conf,0.947198


In [90]:
# Atribuir apenas valores inteiros aos indicadores de tentativas
df_predict_conf.prediction_try = df_predict_conf.prediction_try.apply(get_prediction_index)

In [91]:
df_predict_conf.sample(5)

,tweet_id,jpg_url,prediction_try,confidence
3061,716285507865542656,https://pbs.twimg.com/media/CfDB3aJXEAAEZNv.jpg,2,0.196769
1055,714957620017307648,https://pbs.twimg.com/media/CewKKiOWwAIe3pR.jpg,1,0.251516
3758,836260088725786625,https://pbs.twimg.com/media/C5r-G2IUwAA6KBY.jpg,2,0.078267
3563,801167903437357056,https://pbs.twimg.com/media/Cx5R8wPVEAALa9r.jpg,2,0.061604
1660,819004803107983360,https://pbs.twimg.com/media/C12whDoVEAALRxa.jpg,1,0.351308


In [92]:
df_predict_conf.shape

(5982, 4)

<a id="limpeza:image-prediction:dog"></a>

#### 3.1.3. [`[p1_dog, p2_dog, p3_dog]`, para `is_a_dog`](#limpeza:image-prediction:dog)

In [93]:
# Movendo o resultado da predição, `p#_dog` para `is_a_dog`
df_predict_dog = pd.melt(df_predict_clean,
                        id_vars=['tweet_id', 'jpg_url'],
                        value_vars=['p1_dog', 'p2_dog', 'p3_dog'],
                        var_name=['prediction_try'],
                        value_name='is_a_dog')

In [94]:
df_predict_dog.sample(5)

,tweet_id,jpg_url,prediction_try,is_a_dog
3205,743253157753532416,https://pbs.twimg.com/media/ClCQzFUUYAA5vAu.jpg,p2_dog,True
2504,676215927814406144,https://pbs.twimg.com/media/CWJmzNsWUAE706Z.jpg,p2_dog,False
1423,774639387460112384,https://pbs.twimg.com/media/CsASZqRW8AA3Szw.jpg,p1_dog,True
3250,748699167502000129,https://pbs.twimg.com/media/CmPp5pOXgAAD_SG.jpg,p2_dog,True
1975,888554962724278272,https://pbs.twimg.com/media/DFTH_O-UQAACu20.jpg,p1_dog,True


In [95]:
# Atribuir apenas valores inteiros aos indicadores de tentativas
df_predict_dog.prediction_try = df_predict_dog.prediction_try.apply(get_prediction_index)

In [96]:
df_predict_dog.sample(5)

,tweet_id,jpg_url,prediction_try,is_a_dog
5497,790337589677002753,https://pbs.twimg.com/media/CvfX2AnWYAAQTay.jpg,3,True
5159,737322739594330112,https://pbs.twimg.com/media/Cjt_Hm6WsAAjkPG.jpg,3,False
5310,757393109802180609,https://pbs.twimg.com/media/CoLNAq6WAAAkmdJ.jpg,3,True
4623,681261549936340994,https://pbs.twimg.com/media/CXRTw_5WMAAUDVp.jpg,3,True
5887,870308999962521604,https://pbs.twimg.com/media/DBP1asiUAAEKZI5.jpg,3,True


In [97]:
df_predict_dog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5982 entries, 0 to 5981
Data columns (total 4 columns):
tweet_id          5982 non-null int64
jpg_url           5982 non-null object
prediction_try    5982 non-null int64
is_a_dog          5982 non-null bool
dtypes: bool(1), int64(2), object(1)
memory usage: 146.1+ KB


<a id="limpeza:image-prediction:merge"></a>

#### 3.2. [Unir os dataframes de prediction](#limpeza:image-prediction:merge)

In [125]:
# Merge
df_predict_clean = pd.merge(df_predict_try, df_predict_conf, how='left', on=['tweet_id', 'jpg_url', 'prediction_try'])

In [127]:
df_predict_clean.shape

(5982, 5)

In [126]:
df_predict_clean.sample(5)

,tweet_id,jpg_url,prediction_try,prediction,confidence
5467,784826020293709826,https://pbs.twimg.com/media/CuRDF-XWcAIZSer.jpg,3,irish setter,0.077456
5774,841314665196081154,https://pbs.twimg.com/ext_tw_video_thumb/84131...,3,bloodhound,0.026565
4054,667176164155375616,https://pbs.twimg.com/media/CUJJLtWWsAE-go5.jpg,3,toy poodle,0.106014
2261,670815497391357952,https://pbs.twimg.com/media/CU83IZ8W4AEIh4y.jpg,2,staffordshire bullterrier,0.073430
2113,668204964695683073,https://pbs.twimg.com/media/CUXw3qHWoAAk8HJ.jpg,2,golden retriever,0.107884


In [128]:
df_predict_clean = pd.merge(df_predict_clean, df_predict_dog, how='left', on=['tweet_id', 'jpg_url', 'prediction_try'])

In [130]:
df_predict_clean.shape

(5982, 6)

**Teste: Verificar Merge dos dataframes de predição**

In [131]:
df_predict_clean.sample(5)

,tweet_id,jpg_url,prediction_try,prediction,confidence,is_a_dog
2810,692901601640583168,https://pbs.twimg.com/media/CZ2uU37UcAANzmK.jpg,2,cocker spaniel,0.135164,True
1572,801958328846974976,https://pbs.twimg.com/media/CyEg2AXUsAA1Qpf.jpg,1,staffordshire bullterrier,0.327887,True
5463,783821107061198850,https://pbs.twimg.com/media/CuCxIzyWEAQTnQA.jpg,3,standard poodle,0.133534,True
197,669661792646373376,https://pbs.twimg.com/media/CUsd2TfWwAAmdjb.jpg,1,weasel,0.262802,False
1360,762316489655476224,https://pbs.twimg.com/media/CpRKzZKWAAABGh7.jpg,1,african grey,0.270468,False


------
<a id="limpeza:twitter-api"></a>

## Limpeza: Twitter API

In [132]:
# Cópia do dataframe para limpeza
df_api_clean = df_tweets_api.copy()

In [137]:

df_api_clean.sample(3)

,id,created_at,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,retweet_count,favorite_count,media_url,retweeted,favorited
1713,667866724293877760,Sat Nov 21 00:46:50 +0000 2015,NaN,NaN,False,1041,3008,http://pbs.twimg.com/media/CUS9PlUWwAANeAD.jpg,False,False
957,697270446429966336,Wed Feb 10 04:06:43 +0000 2016,NaN,NaN,False,1965,4858,http://pbs.twimg.com/media/Ca0zxGjW8AEfyYl.jpg,False,False
492,757597904299253760,Mon Jul 25 15:26:30 +0000 2016,NaN,NaN,False,313,0,http://pbs.twimg.com/media/CoOGZjiWAAEMKGx.jpg,False,False


<a id="limpeza:tweets-api:columns"></a>

### 1. [Os nomes das colunas `id` e `media_url` estão fora do padrão dos demais dataframes, respectivamente `tweet_id` e `jpg_url` ](#limpeza:tweets-api:columns)

In [143]:
df_api_clean.rename(columns={'id':'tweet_id', 'media_url': 'jpg_url'}, inplace=True)

**Teste: Verificar colunas renomeadas**

In [144]:
df_api_clean.columns

Index(['tweet_id', 'created_at', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'is_quote_status', 'retweet_count',
       'favorite_count', 'jpg_url', 'retweeted', 'favorited'],
      dtype='object')

<a id="limpeza:tweets-api:duplicated"></a>

### 2. [Diferentes Tweets referenciando a mesma imagem, duplicidade](#limpeza:tweets-api:duplicated)

Assim como na [limpeza de duplicidade de imagens no dataframe de predições](#limpeza:image-prediction:duplicated-urls), é importante de remover os tweets não originais, os retweets. Com auxílio do dataframe `df_arch_clean`, que possui somente referências a tweets originais, será possível identifica-los e descartar os tweets indesejados.

In [138]:
df_api_clean.shape

(1820, 10)

In [146]:
df_api_clean = df_api_clean.loc[df_api_clean.tweet_id.isin(df_arch_clean.tweet_id)]

In [147]:
df_api_clean.shape

(1746, 10)

Ao remover os ids que referenciavam a **retweets**, houve uma redução de 74 registros, de **1820** para **1746**

<a id="limpeza:tweets-api:datetime"></a>

### 3. [Data de criação (`created_at`) como `string`](#limpeza:tweets-api:datetime)
* Converter datas em string para o formato `datetime`

In [149]:
df_api_clean.created_at = pd.to_datetime(df_api_clean.created_at)

**Teste: Validar tipo de variável em `created_at`**

In [150]:
df_api_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1746 entries, 0 to 1819
Data columns (total 10 columns):
tweet_id                 1746 non-null int64
created_at               1746 non-null datetime64[ns, UTC]
in_reply_to_status_id    22 non-null float64
in_reply_to_user_id      22 non-null float64
is_quote_status          1746 non-null bool
retweet_count            1746 non-null int64
favorite_count           1746 non-null int64
jpg_url                  1746 non-null object
retweeted                1746 non-null bool
favorited                1746 non-null bool
dtypes: bool(3), datetime64[ns, UTC](1), float64(2), int64(3), object(1)
memory usage: 114.2+ KB


In [151]:
df_api_clean.sample(5)

,tweet_id,created_at,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,retweet_count,favorite_count,jpg_url,retweeted,favorited
1456,672877615439593473,2015-12-04 20:38:19+00:00,NaN,NaN,False,376,1043,http://pbs.twimg.com/media/CVaKn75XAAEU09u.jpg,False,False
151,819004803107983360,2017-01-11 02:15:36+00:00,NaN,NaN,False,39753,91537,http://pbs.twimg.com/media/C12whDoVEAALRxa.jpg,False,False
488,757741869644341248,2016-07-26 00:58:34+00:00,NaN,NaN,False,3480,7223,http://pbs.twimg.com/media/CoQKNY7XYAE_cuX.jpg,False,False
879,704113298707505153,2016-02-29 01:17:46+00:00,NaN,NaN,False,594,1923,http://pbs.twimg.com/media/CcWDTgOUcAA7Ia9.jpg,False,False
1199,680494726643068929,2015-12-25 21:06:00+00:00,NaN,NaN,False,510,1785,http://pbs.twimg.com/media/CXGaVxOWAAADjhF.jpg,False,False


<a id="limpeza:tweets-api:info"></a>

### 4. [Valor do id na coluna `in_reply_to_status_id` e `in_reply_to_user_id` como `float`](#limpeza:tweets-api:info)
* Converter para Inteiro as colunas `in_reply_to_status_id` e `in_reply_to_user_id` e atribuir np.nan as que estiverem sem valor

In [156]:
df_api_clean.in_reply_to_status_id.value_counts()

8.558181e+17    1
6.717299e+17    1
6.747934e+17    1
6.747522e+17    1
7.468859e+17    1
6.706684e+17    1
6.844811e+17    1
6.749998e+17    1
6.842229e+17    1
7.291135e+17    1
6.753494e+17    1
6.754971e+17    1
6.715610e+17    1
7.030419e+17    1
6.757073e+17    1
7.590995e+17    1
6.914169e+17    1
7.032559e+17    1
6.678065e+17    1
6.671522e+17    1
6.920419e+17    1
7.331095e+17    1
Name: in_reply_to_status_id, dtype: int64

------
<a id="armazenamento"></a>

# Armazenamento

# Relatórios

* Data wrangling efforts
* Analyses and visualizations